# SKLearn Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to deploy iris classifier based on Scikit Learn model using MLP 

In [65]:
!cat requirements.txt

scikit-learn>=1.1.2
# caraml
joblib>=0.13.0,<1.2.0  # >=1.2.0 upon upgrade of kserve's version
cloudpickle==2.0.0

In [1]:
!pip list | grep caram

caraml-auth-google        0.0.0.post7
caraml_sdk                0.0.0.post73.dev0
caraml-upi-protos         1.0.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip3 install --upgrade -r requirements.txt > /dev/null


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
# import basic
import warnings
import os

# Import modeling lib
from sklearn import svm
from sklearn import datasets
from joblib import dump
from sklearn.datasets import load_iris

# Load merlin SDK
import caraml
from caraml.client.caraml_client import CaraMLClient
from caraml.models.model import ModelType
# warnings.filterwarnings('ignore')

/Users/kuppilpawanmaniteja/Documents/github/venv-caraml-sdk-test/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/kuppilpawanmaniteja/Documents/github/venv-caraml-sdk-test/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_endpoint_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/kuppilpawanmaniteja/Documents/github/venv-caraml-sdk-test/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_prediction_output" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/kuppilpaw

In [2]:
import logging
logging.basicConfig()
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.DEBUG)


## 1. Initialize MLP Resources

### 1.1 Set MLP Server

In [3]:
# Set MLP Server
CARAML_URL = os.environ.get("CARAML_URL", "http://console.d.ai.golabs.io")
caraml = CaraMLClient(caraml_url=CARAML_URL, use_google_oauth=True)

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
DEBUG:caraml_client:Registered 3 from MLP. methods: ['get_project', 'list_projects', 'set_project']
DEBUG:caraml_client:Registered 12 from Models. methods: ['deploy', 'get_default_environment', 'get_environment', 'get_model', 'get_or_create_model', 'get_or_create_project', 'get_project', 'list_environment', 'list_project', 'new_model_version', 'standard_transformer_simulate', 'undeploy']
DEBUG:caraml_client:setting method name: get_projec

In [4]:
# Print MLP Server URL
print("Caraml Server URL: {}".format(CARAML_URL))

Caraml Server URL: http://console.d.ai.golabs.io


### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [5]:
caraml.set_project("teja-test-project")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): console.d.ai.golabs.io:80
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/v1/projects?name=teja-test-project HTTP/1.1" 200 291


In [6]:
caraml.list_environment()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): console.d.ai.golabs.io:80
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/environments HTTP/1.1" 200 1559


### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in MLP is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by MLP are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [7]:
model = caraml.get_or_create_model("sklearn-sample", project_name="teja-test-project", model_type= ModelType.SKLEARN)

DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/v1/projects?name=teja-test-project HTTP/1.1" 200 291
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/projects/129/models?name=sklearn-sample HTTP/1.1" 200 264


## 2. Train and Deploy

### 2.1 Train and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()
```

In [8]:
model_dir = "sklearn-model"
MODEL_FILE = "model.joblib"

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

# Save model to local directory
dump(clf, os.path.join(model_dir, MODEL_FILE))


['sklearn-model/model.joblib']

In [9]:
# Create new version of the model
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()

DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "POST /api/merlin/v1/models/360/versions HTTP/1.1" 201 691
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): mlflow.d.ai.golabs.io:443
DEBUG:urllib3.connectionpool:https://mlflow.d.ai.golabs.io:443 "GET /api/2.0/mlflow/runs/get?run_uuid=2f48073d9da9449aabb46f8d2bb992b4&run_id=2f48073d9da9449aabb46f8d2bb992b4 HTTP/1.1" 200 406
DEBUG:urllib3.connectionpool:https://mlflow.d.ai.golabs.io:443 "GET /api/2.0/mlflow/experiments/get?experiment_id=2976 HTTP/1.1" 200 190
DEBUG:urllib3.connectionpool:https://mlflow.d.ai.golabs.io:443 "GET /api/2.0/mlflow/runs/get?run_uuid=2f48073d9da9449aabb46f8d2bb992b4&run_id=2f48073d9da9449aabb46f8d2bb992b4 HTTP/1.1" 200 406
DEBUG:urllib3.connectionpool:https://mlflow.d.ai.golabs.io:443 "POST /api/2.0/mlflow/runs/update HTTP/1.1" 200 359
DEBUG:urllib3.connectionpool:https://mlflow.d.ai.golabs.io:443 "GET /api/2.0/mlflow/runs/get?run_uuid=2f48073d9da9449aabb46f8d2bb992b4&run_id=2f48073d9

## 2.2 Deploy Model

In [10]:
endpoint = caraml.deploy(v)

DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/environments HTTP/1.1" 200 1559
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "POST /api/merlin/v1/models/360/versions/6/endpoint HTTP/1.1" 201 1658
Deploying model sklearn-sample version 6
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint/bfda523c-cae5-46a8-9bea-06416260bbdd HTTP/1.1" 200 1667
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint/bfda523c-cae5-46a8-9bea-06416260bbdd HTTP/1.1" 200 1667
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint/bfda523c-cae5-46a8-9bea-06416260bbdd HTTP/1.1" 200 1667
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint/bfda523c-cae5-46a8-9bea-06416260bbdd HTTP/1.1" 200 1667
0% [#                             ] 

Model sklearn-sample version 6 is deployed.
View model version logs: http://console.d.ai.golabs.io/merlin/projects/129/models/360/versions/6/endpoints/bfda523c-cae5-46a8-9bea-06416260bbdd/logs


DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint HTTP/1.1" 200 1855


## 2.3 Send Test Request

In [11]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

DEBUG:asyncio:Using selector: KqueueSelector
Note: Unnecessary use of -X or --request, POST is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.203.170.169:80...
* Connected to sklearn-sample-6.teja-test-project.models.id.d.gods.golabs.io (10.203.170.169) port 80 (#0)
> POST /v1/models/sklearn-sample-6:predict HTTP/1.1
> Host: sklearn-sample-6.teja-test-project.models.id.d.gods.golabs.io
> User-Agent: curl/8.1.2
> Accept: */*
> Content-Length: 81
> Content-Type: application/x-www-form-urlencoded
> 
} [81 bytes data]
< HTTP/1.1 200 OK
< content-length: 21
< content-type: application/json
< date: Mon, 04 Mar 2024 09:54:04 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 13
< 
{ [21 bytes data]
100   102  100    21  100    81     85    329 --:--:-- --:--:-- --:--:

{"predictions":[2,2]}

## 3.4 Delete Deployment

In [12]:
caraml.undeploy(v)

DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/environments HTTP/1.1" 200 1559
DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "GET /api/merlin/v1/models/360/versions/6/endpoint HTTP/1.1" 200 1855


Deleting deployment of model sklearn-sample version 6 from enviroment id-dev


DEBUG:urllib3.connectionpool:http://console.d.ai.golabs.io:80 "DELETE /api/merlin/v1/models/360/versions/6/endpoint/bfda523c-cae5-46a8-9bea-06416260bbdd HTTP/1.1" 200 0
